# Tutorial 
from https://docs.sqlalchemy.org/en/13/core/tutorial.html

In [1]:
import sqlalchemy

In [4]:
from sqlalchemy import create_engine

We use an in-memory SQLite database, which is the easiest way to test things without having a database defined anywhere

In [3]:
engine = create_engine("sqlite:///:memory", echo=True)

The `echo` flag is a shortcut to setting up SQLAlchemy logging, using python's `logging` module. With it enabled, we'll see all the generated SQL produced.

The return value of `create_engine()` is an instance of `Engine` and it represents the core interface to the database, adapted through a dialect that handles the details of the database and DBAPI in use. The SQLite dialect will interpret instructions to python's `sqlite` module.

The first time a method like `Engine.execute()` or `Engine.connect()` is called, the `Engine` establishes a real DBAPI connection to the database, used to emit the SQL.

<b>Dictionary:</b> <i>DBAPI</i> is shorthand for the phrase "Python Database API Specification" which is a widely used specification within python to define common usage patterns for database connection packages. 
SQLAlchemy's `dialect` system is constructed around the operation of DBAPI. 

For example, the `create_engine()` URL `postgresql+psycopg2://@localhost/test` refers to the `psycopg2` DBAPI/dialect combination, whereas the URL `mysql+mysqldb://@localhost/test` refers to the `MySQL for Python` DBAPI/dialect combination

# Defining and creating tables

SQL constructs expressions against table columns. In SQLAlchemy, a column is represented by an object called `Column`, and in all cases a `Column` is associated with a `Table`. A collection of `Table` objects and their associated child objects is referred to as <i>database metadata</i>.

Here we will explicitly lay out `Table` objects, but SA can "import" whole sets of `Table` objects automatically from an existing database (<i>table reflection</i>).

We define our tables within a catalog called `MetaData` using the `Table` construct, which resembles regular `SQL CREATE TABLE` statements. 

We will make two tables, one which represents "users" in an application, and another which represents 0 or more "email addresses" for each row in the "users" table.

In [6]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [7]:
metadata = MetaData()

In [8]:
help(Table)

Help on class Table in module sqlalchemy.sql.schema:

class Table(sqlalchemy.sql.base.DialectKWArgs, SchemaItem, sqlalchemy.sql.selectable.TableClause)
 |  Table(*args, **kw)
 |  
 |  Represent a table in a database.
 |  
 |  e.g.::
 |  
 |      mytable = Table("mytable", metadata,
 |                      Column('mytable_id', Integer, primary_key=True),
 |                      Column('value', String(50))
 |                 )
 |  
 |  The :class:`_schema.Table`
 |  object constructs a unique instance of itself based
 |  on its name and optional schema name within the given
 |  :class:`_schema.MetaData` object. Calling the :class:`_schema.Table`
 |  constructor with the same name and same :class:`_schema.MetaData` argument
 |  a second time will return the *same* :class:`_schema.Table`
 |  object - in this way
 |  the :class:`_schema.Table` constructor acts as a registry function.
 |  
 |  .. seealso::
 |  
 |      :ref:`metadata_describing` - Introduction to database metadata
 |  
 |  C

In [9]:
users = Table(
"users", 
metadata,
Column("id", Integer, primary_key=True),
Column("name", String),
Column("fullname", String))

In [11]:
addresses = Table(
"addresses",
metadata, 
Column("id", Integer, primary_key=True),
Column("user_id", None, ForeignKey("users.id")),
Column("email_address", String, nullable=False))

To tell the `MetaData` we'd like to create our selection of tables inside the SQLite database, we use `create_all()` passing it to the `engine` which points to our database. This will check for the presence of each table first before creating, so it's safe to call multiple times.

In [12]:
metadata.create_all(engine)

2021-02-10 11:54:50,641 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-10 11:54:50,645 INFO sqlalchemy.engine.base.Engine ()
2021-02-10 11:54:50,647 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-10 11:54:50,648 INFO sqlalchemy.engine.base.Engine ()
2021-02-10 11:54:50,650 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2021-02-10 11:54:50,653 INFO sqlalchemy.engine.base.Engine ()
2021-02-10 11:54:50,656 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2021-02-10 11:54:50,658 INFO sqlalchemy.engine.base.Engine ()
2021-02-10 11:54:50,659 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2021-02-10 11:54:50,661 INFO sqlalchemy.engine.base.Engine ()
2021-02-10 11:54:50,663 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2021-02-10 11:54:50,664 INFO sqlalchemy.engine.base.Engine ()
2021-02-10 11:54:5

Here we generated the VARCHAR columns without a length - on SQLite and PostgreSQL this is a valid datatype, but on others, it's not allowed. A "length" can be provided to the `String` type as follows:

In [13]:
Column("name", String(50))

Column('name', String(length=50), table=None)

Firebird and Oracle require sequences to generate new primary key identifiers: 

In [14]:
from sqlalchemy import Sequence

In [15]:
Column("id", Integer, Sequence("user_id_seq"), primary_key=True)

Column('id', Integer(), table=None, primary_key=True, nullable=False, default=Sequence('user_id_seq'))

A full, foolproof `Table` is therefore: 
```
users_max = Table("users", metadata,
             Column("id", Integer, Sequence("user_id_seq"),
                   primary_key=True),
              Column("name", String(50)),
              Column("fullname", String(50)),
              Column("nickname", String(50))
             )
```

This just illustrates the difference between minimal constructs geared towards in-python usage vs one that will be used to emit CREATE TABLE statements on a set of backends with more stringent requirements.

# Insert Expressions

The first SQL expression we'll create is the `Insert` construct, which represents an INSERT statement. This is typically created relative to its target table: 

In [18]:
ins = users.insert()

To see a sample of the SQL this construct produces, use the `str()` function:

In [19]:
str(ins)

'INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)'

Note that the INSERT statement names every column in the `users` table. This can be limited using the `values()` method which establishes the VALUES clause of the INSERT explicitly: 

In [20]:
ins = users.insert().values(name="jack", fullname="Jack Jones")

In [21]:
str(ins)

'INSERT INTO users (name, fullname) VALUES (:name, :fullname)'

In [22]:
users

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None)

The data we placed in `values` doesn't get rendered into the string - we just get named bind paramters.

The data is stored in the `Insert` construct, but only comes out when the data is executed. Since the data consists of literal values, SQLAlchemy automatically generates bind parameters for them.

We can look at the data for now by looking at the compiled form of the statement:

In [24]:
ins.compile().params

{'name': 'jack', 'fullname': 'Jack Jones'}

# Executing

The interesting part of an `Insert` is executing it. We will focus on the most explicit method of executing an SQL construct and later touch on some shortcut ways to do it.

The `engine` obj we created is a repository for database connections capable of issuing SQL to the database. To acquire a connection, we will use the `Engine.connect()` method: 

In [25]:
conn = engine.connect()

In [26]:
conn

The `Connection` object represents an actively checked out DBAPI connection resource. 

In [27]:
result = conn.execute(ins)

2021-02-10 15:38:10,294 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2021-02-10 15:38:10,297 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Jones')
2021-02-10 15:38:10,301 INFO sqlalchemy.engine.base.Engine COMMIT


The INSERT statement is now issued to the database. 

note we did get some question mark bind parameters (?) in the output instead of named parameters. 

This is because when executed, the `Connection` used the SQLite dialect to help generate the statement, and the statement isn't aware of the dialect. We can view this manually as follows: 

In [28]:
ins.bind = engine

In [29]:
str(ins)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

What about the `result` value we got when we called `execute()`?

Because the SQLAlchemy `Connection` obj references a DBAPI connection, the result, known as a `ResultProxy` object is analogous to the DBAPI cursor object. 

In the case of an INSERT, we can get important info from it, like the primary key values which were generated from our statement: 

In [30]:
result.inserted_primary_key

[1]

The value of 1 was automatically generated by SQLite because we did not specify the `id` column in our `Insert` statement.

# Executing Multiple Statements

Let's create a generic `Insert` statement in the "normal" way:

In [31]:
ins = users.insert()

In [32]:
conn.execute(ins, id=2, name="wendy", fullname="Wendy Williams")

2021-02-10 15:43:30,610 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2021-02-10 15:43:30,617 INFO sqlalchemy.engine.base.Engine (2, 'wendy', 'Wendy Williams')
2021-02-10 15:43:30,623 INFO sqlalchemy.engine.base.Engine COMMIT


Now execute many by sending in a list of dictionaries containing a set of parameters to be inserted

In [33]:
conn.execute(addresses.insert(), [
    {"user_id":1, "email_address":"jack@yahoo.com"},
    {"user_id":1, "email_address":"jack@msn.com"},
    {"user_id":2, "email_address":"www@www.org"},
    {"user_id":2, "email_address":"wendy@aol.com"}
    
])

2021-02-10 15:45:55,430 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2021-02-10 15:45:55,437 INFO sqlalchemy.engine.base.Engine ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2021-02-10 15:45:55,450 INFO sqlalchemy.engine.base.Engine COMMIT


When executing many parameters, the dictionaries must have the same keys - some can't have more/less keys than others because the `Insert` statement is compiled against the first dictionary address in the list assuming the following ones are compatible with that statement

# Selecting

Now we have inserted some data, let's select it:

In [34]:
from sqlalchemy.sql import select

In [35]:
s = select([users])

In [37]:
result = conn.execute(s)

2021-02-10 15:47:58,874 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-02-10 15:47:58,879 INFO sqlalchemy.engine.base.Engine ()


The result returned is a `ResultProxy` object, which acts like a DBAPI cursor, including methods like `fetchone()` and `fetchall()`. 

These methods return row objects which are provided via the `RowProxy` class. 

The result object can be iterated directly to provide an iterator of `RowProxy` objects: 

In [38]:
for row in result:
    print(row)

(1, 'jack', 'Jack Jones')
(2, 'wendy', 'Wendy Williams')


Can either retrieve data using string names of columns: 

In [39]:
result = conn.execute(s)

2021-02-10 15:51:10,235 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-02-10 15:51:10,236 INFO sqlalchemy.engine.base.Engine ()


In [40]:
row = result.fetchone()

In [42]:
print("name:", row["name"], "fullname:", row["fullname"])

name: jack fullname: Jack Jones


Or using integer indexing:

In [44]:
# this is an iterator - goes to next one 
row = result.fetchone()
print("name", row[1], "fullname", row[2])

name wendy fullname Wendy Williams


Or use the SQL construct that directly corresponds to a particular column as the mapping key - use the column objects selected in our SELECT directly as keys: 

In [45]:
for row in conn.execute(s):
    print('name:', row[users.c.name], "fullname", row[users.c.fullname])

2021-02-10 15:54:17,471 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-02-10 15:54:17,480 INFO sqlalchemy.engine.base.Engine ()
name: jack fullname Jack Jones
name: wendy fullname Wendy Williams


We can also specify these columns when using SELECT: 

In [46]:
s = select([users.c.name, users.c.fullname])

In [47]:
result = conn.execute(s)

2021-02-10 15:55:47,870 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2021-02-10 15:55:47,872 INFO sqlalchemy.engine.base.Engine ()


In [48]:
for row in result: 
    print(row)

('jack', 'Jack Jones')
('wendy', 'Wendy Williams')
